#ChatBot_Project

In [1]:
import nltk
import json
import numpy as np
import random
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split

punkt is a pre-trained model included in the Natural Language Toolkit (nltk) that is used for tokenization. Tokenization is the process of splitting text into individual units, such as words or sentences

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

wordnet is used to download the WordNet corpus, which is necessary for performing lemmatization with NLTK's

In [3]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# Data preprocessing

In [4]:
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']
lemmatizer = WordNetLemmatizer()

In [5]:
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [6]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [7]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))


In [8]:
print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

34 documents
8 classes ['about', 'chatbot', 'connect', 'goodbye', 'greeting', 'location', 'movies', 'thanks']
68 unique lemmatized words ["'s", 'Awesome', 'Bye', 'Chatbot', 'Give', 'Good', 'Goodbye', 'Hello', 'Hey', 'Hi', 'Hola', 'How', 'Is', 'Nice', 'Recommend', 'See', 'Suggest', 'Tell', 'Thank', 'Thanks', 'That', 'Till', 'What', 'Where', 'Which', 'Who', 'Yourself', 'about', 'account', 'address', 'any', 'anyone', 'are', 'built', 'bye', 'can', 'chatbot', 'chatting', 'connect', 'day', 'favourite', 'for', 'helpful', 'helping', 'i', 'is', 'later', 'link', 'located', 'location', 'me', 'medium', 'movie', 'name', 'next', 'out', 'reach', 'social', 'some', 'thanks', 'there', 'this', 'time', 'to', 'way', 'we', 'you', 'your']


#Training Data

In [9]:
training = []
empty_array = [0] * len(classes)

In [10]:
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row = list(empty_array)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [11]:
random.shuffle(training)
trainings = np.array(training, dtype=object)

In [12]:
# Split data into training and testing sets
train_x, test_x, train_y, test_y = train_test_split(list(trainings[:, 0]), list(trainings[:, 1]), test_size=0.2, random_state=42)

#Model creating

In [13]:
# Build the model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [14]:
# Compile the model
sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Train the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=150, batch_size=5, verbose=1)


Epoch 1/150
6/6 [==============================] - 1s 4ms/step - loss: 2.0885 - accuracy: 0.1481
Epoch 2/150
6/6 [==============================] - 0s 3ms/step - loss: 2.0598 - accuracy: 0.2593
Epoch 3/150
6/6 [==============================] - 0s 3ms/step - loss: 1.9989 - accuracy: 0.3333
Epoch 4/150
6/6 [==============================] - 0s 4ms/step - loss: 2.0308 - accuracy: 0.2963
Epoch 5/150
6/6 [==============================] - 0s 3ms/step - loss: 1.9685 - accuracy: 0.3704
Epoch 6/150
6/6 [==============================] - 0s 3ms/step - loss: 1.9374 - accuracy: 0.3333
Epoch 7/150
6/6 [==============================] - 0s 3ms/step - loss: 1.8552 - accuracy: 0.3333
Epoch 8/150
6/6 [==============================] - 0s 4ms/step - loss: 1.8643 - accuracy: 0.4815
Epoch 9/150
6/6 [==============================] - 0s 3ms/step - loss: 1.8159 - accuracy: 0.3333
Epoch 10/150
6/6 [==============================] - 0s 3ms/step - loss: 1.7517 - accuracy: 0.4815
Epoch 11/150
6/6 [===========

#END